___

<p style="text-align: center;"><img src="https://docs.google.com/uc?id=1lY0Uj5R04yMY3-ZppPWxqCr5pvBLYPnV" class="img-fluid" alt="CLRSWY"></p>

___

# WELCOME!

Welcome to "***Car Price Prediction Project***". This is the first medium project of ***Machine Learning*** course. In this project you will have the opportunity to apply many algorithms commonly used for regression problems.

Also, you will apply various processes such as pre-processing, ***train-test spilit*** and ***cross validation*** that you will use in algorithm modeling and prediction processes in Python with ***scikit-learn***. 

Before diving into the project, please take a look at the determines and tasks.

- **NOTE:** This project assumes that you already know the basics of coding in Python. You should also be familiar with the theory behind regression algorithms and scikit-learn module as well as Machine Learning before you begin.


# #Determines

**Auto Scout** data which using for this project, scraped from the on-line car trading company(https://www.autoscout24.com)in 2019, contains many features of 9 different car models. In this project, you will use the data set which is already preprocessed and prepared for algorithms .

The aim of this project to understand of machine learning algorithms. Therefore, you will not need any EDA process as you will be working on the edited data.

---

In this Senario, you will estimate the prices of cars using regression algorithms.

While starting you should import the necessary modules and load the data given as pkl file. Also you'll need to do a few pre-processing before moving to modelling. After that you will implement ***Linear Regression, Ridge Regression, Lasso Regression,and Elastic-Net algorithms respectively*** (After completion of Unsupervised Learning section, you can also add bagging and boosting algorithms such as ***Random Forest and XG Boost*** this notebook to develop the project. You can measure the success of your models with regression evaluation metrics as well as with cross validation method.

For the better results, you should try to increase the success of your models by performing hyperparameter tuning. Determine feature importances for the model. You can set your model with the most important features for resource saving. You should try to apply this especially in Random Forest and XG Boost algorithms. Unlike the others, you will perform hyperparameter tuning for Random Forest and XG Boost using the ***GridSearchCV*** method. 

Finally You can compare the performances of algorithms, work more on the algorithm have the most successful prediction rate.






# #Tasks

#### 1. Import Modules, Load Data and Data Review
#### 2. Data Pre-Processing
#### 3. Implement Linear Regression 
#### 4. Implement Ridge Regression
#### 5. Implement Lasso Regression 
#### 6. Implement Elastic-Net
#### 7. Visually Compare Models Performance In a Graph

## 1. Import Modules, Load Data and Data Review

In [3]:
import pandas as pd      
import numpy as np 
#import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

from scipy.stats import skew

from sklearn.model_selection import cross_validate, cross_val_score
import warnings
warnings.filterwarnings('ignore')
plt.rcParams["figure.figsize"] = (10,6)
pd.set_option('display.max_columns', 500) # 500 sütuna kadar göster
pd.set_option('display.max_rows', 500) # 500 satıra kadar göster

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
df = pd.read_csv("final_scout_not_dummy.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df2 = df.copy()

## Feature Engineering

In [ ]:
df.select_dtypes(include ="object").head()   # sadece object sütunları getiriyoruz

In [ ]:
for col in df.select_dtypes('object'):
    print(f"{col:<20}:", df[col].nunique())

In [ ]:
df.make_model.value_counts() # Audi A2  1 tane olduğundan atabiliriz modele etkisi yok

In [ ]:
ax = df.make_model.value_counts().plot(kind ="bar")
ax.spines['top'].set_visible(False) # tepe çzgisini kaldırdık
ax.spines['right'].set_visible(False) # sağ çzigiyi kaldırdık
#ax.axis("off")
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.03, p.get_height() * 1.03)) #üzerlerine ağırlıkları yazdırdık

In [ ]:
df[df.make_model=="Audi A2"]

In [ ]:
df.drop(index=[2614], inplace =True) # Audi A2 yi index numarasına göre drop ettik

In [ ]:
df.shape

In [ ]:
sns.histplot(df.price, bins=50, kde=True) # price 4000$ üstü verin,n normal dağılımını bozduğu görülüyor

In [ ]:
skew(df.price)

In [ ]:
df_numeric = df.select_dtypes(include ="number") # numeric değerleri aldık
df_numeric

In [ ]:
sns.heatmap(df_numeric.corr(), annot =True)

## multicollinearity control

In [ ]:
df_numeric.corr()[(df_numeric.corr()>= 0.9) & (df_numeric.corr() < 1)].any().any()

In [ ]:
df_numeric.corr()[(df_numeric.corr()<= -0.9) & (df_numeric.corr() > -1)].any().any()

In [ ]:
sns.boxplot(df.price) # 4000$ üstü outlier görünüyor ama araba grupları için ayrı ayrı bakamak daha sağlıklı, bu bilgi verinin tamamı için geçerlş

In [ ]:
plt.figure(figsize=(16,6))
sns.boxplot(x="make_model", y="price", data=df, whis=3) # Audi A3, Opel Astra,Opel Insignia,Renault Clio ve Renault Espace de outlier veri görünüyor diğerlerinde yok 
plt.show()

## Get dummies 

In [ ]:
#bu sütunlardaki verileri virgülle ayırıp, get dummies yapıp  sütunların başınada kısaltmalarını yazıyor ve df e ekliyor
df = df.join(df["Comfort_Convenience"].str.get_dummies(sep = ",").add_prefix("cc_")) 
df = df.join(df["Entertainment_Media"].str.get_dummies(sep = ",").add_prefix("em_"))
df = df.join(df["Extras"].str.get_dummies(sep = ",").add_prefix("ex_"))
df = df.join(df["Safety_Security"].str.get_dummies(sep = ",").add_prefix("ss_"))

In [ ]:
df.drop(["Comfort_Convenience","Entertainment_Media","Extras","Safety_Security"], axis=1, inplace=True)

In [ ]:
df = pd.get_dummies(df, drop_first =True) # ilk sütunları düşürdük fazla ram fazla cpu olmasın diye

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().any().any()

In [ ]:
corr_by_price = df.corr()["price"].sort_values()[:-1]
corr_by_price

In [ ]:
plt.figure(figsize = (20,10))
sns.barplot(x = corr_by_price.index, y = corr_by_price)
plt.xticks(rotation=90)
plt.tight_layout();

## 2. Data Pre-Processing

As you know, the data set must be edited before proceeding to the implementation of the model. As the last step before model fitting, you need to spilit the data set as train and test. Then, you should train the model with train data and evaluate the performance of the model on the test data. You can use the train and test data you have created for all algorithms.

You must also drop your target variable, the column you are trying to predict.

You can use many [performance metrics for regression](https://medium.com/analytics-vidhya/evaluation-metrics-for-regression-problems-343c4923d922) to measure the performance of the regression model you train. You can define a function to view different metric results together.

You can also use the [cross validation](https://towardsdatascience.com/cross-validation-explained-evaluating-estimator-performance-e51e5430ff85) method to measure the estimator performance. Cross validation uses different data samples from your test set and calculates the accuracy score for each data sample. You can calculate the final performance of your estimator by averaging these scores.

### Train | Test Split

In [ ]:
df

In [ ]:
X= df.drop(columns="price")
y= df.price

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

## 3. Implement Linear Regression

 - Import the modul
 - Fit the model 
 - Predict the test set
 - Determine feature coefficiant
 - Evaluate model performance (use performance metrics for regression and cross_val_score)
 - Compare different evaluation metrics
 
*Note: You can use the [dir()](https://www.geeksforgeeks.org/python-dir-function/) function to see the methods you need.*

In [ ]:
def train_val(model, X_train, y_train, X_test, y_test):
    
    y_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train) # overfitting var mı yok mu kıyaslamak için
    
    scores = {"train": {"R2" : r2_score(y_train, y_train_pred),
    "mae" : mean_absolute_error(y_train, y_train_pred),
    "mse" : mean_squared_error(y_train, y_train_pred),                          
    "rmse" : np.sqrt(mean_squared_error(y_train, y_train_pred))},
    
    "test": {"R2" : r2_score(y_test, y_pred),
    "mae" : mean_absolute_error(y_test, y_pred),
    "mse" : mean_squared_error(y_test, y_pred),
    "rmse" : np.sqrt(mean_squared_error(y_test, y_pred))}}
    
    return pd.DataFrame(scores)

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train,y_train)

In [ ]:
pd.options.display.float_format = '{:.3f}'.format #nokta sonrası yorumlama kolay olsun diye sadece 3 basamak aldık

In [ ]:
train_val(lm, X_train, y_train, X_test, y_test)

## Adjusted R2 Score

In [ ]:
# get dummies yaptığımız için sütun sayısı arttı bundan dolayı r2 score da yalancı bir artış olabilir,bakalım
# normalde satır ve sütun sayıları dengeli artmalı
def adj_r2(y_test, y_pred, df):   
    r2 = r2_score(y_test, y_pred) 
    n = df.shape[0]   # number of observations
    p = df.shape[1]-1 # number of independent variables 
    adj_r2 = 1 - (1-r2)*(n-1)/(n-p-1)
    return adj_r2

In [ ]:
y_pred = lm.predict(X_test)

In [ ]:
adj_r2(y_test, y_pred, df) # çok bir değişim olamamış,sıkıntı yok 

## Cross Validate

In [ ]:
model = LinearRegression()
scores = cross_validate(model, X_train, y_train, scoring=['r2', 
            'neg_mean_absolute_error','neg_mean_squared_error','neg_root_mean_squared_error'], cv =10)

In [ ]:
pd.DataFrame(scores)

In [ ]:
pd.DataFrame(scores).iloc[:, 2:].mean()

In [ ]:
train_val(lm, X_train, y_train, X_test, y_test)

In [ ]:
2405/df.price.mean()

## Prediction Error

In [ ]:
#!pip install yellowbrick

In [ ]:
# actual eğri ile predict eğrisi arasındaki açıklık bariz,
# bunu özellikle 30000 sonrası verilerden görüyoruz,bu veriler predict eğrisini uzaklaştırıyor gerçek verilerden
from yellowbrick.regressor import PredictionError
from yellowbrick.features import RadViz

visualizer = RadViz(size=(720, 3000))
model = LinearRegression()
visualizer = PredictionError(model)
visualizer.fit(X_train, y_train)  # Fit the training data to the visualizer
visualizer.score(X_test, y_test)  # Evaluate the model on the test data
visualizer.show();

## Residual Plot

In [ ]:
# 30000 e kadar eğrinin alt ve üstü simetrik, normal dağılım gösteriyor,sonrası predicts altta kalıyor
plt.figure(figsize=(12,8))
residuals = y_test-y_pred

sns.scatterplot(x = y_test, y = -residuals) #-residuals
plt.axhline(y = 0, color ="r", linestyle = "--")
plt.ylabel("residuals")
plt.show()

In [ ]:
from yellowbrick.regressor import ResidualsPlot

visualizer = RadViz(size=(1000, 720))
model = LinearRegression()
visualizer = ResidualsPlot(model)

visualizer.fit(X_train, y_train)  # Fit the training data to the visualizer
visualizer.score(X_test, y_test)  # Evaluate the model on the test data
visualizer.show();       

## Dropping observations from the dataset that worsen my predictions

In [ ]:
df3 = df[~(df.price>35000)] # 35000 sonrasını tahminerimi bozduğundan attık
df3

In [ ]:
len(df[df.price>35000]) # zaten atılan kısım verinin %3 oluşturur, yani küçük bir kısım 

In [ ]:
df2[df2.price>35000].groupby("make_model").count().iloc[:,0]

In [ ]:
df2.make_model.value_counts()

In [ ]:
X = df3.drop(columns = "price")
y = df3.price

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
lm2 = LinearRegression()
lm2.fit(X_train,y_train)

In [ ]:
visualizer = RadViz(size=(720, 3000))
model = LinearRegression()
visualizer = PredictionError(model)
visualizer.fit(X_train, y_train)  # Fit the training data to the visualizer
visualizer.score(X_test, y_test)  # Evaluate the model on the test data
visualizer.show();

In [ ]:
train_val(lm2, X_train, y_train, X_test, y_test)

In [ ]:
1908/df3.price.mean()    #hatam  %13 den %11 düştü tahminleri bozan kısmı atınca

In [ ]:
2405/df2.price.mean()

In [ ]:
y_pred = lm2.predict(X_test)

lm_R2 = r2_score(y_test, y_pred)
lm_mae = mean_absolute_error(y_test, y_pred)
lm_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
my_dict = { 'Actual': y_test, 'Pred': y_pred, 'Residual': y_test-y_pred }
compare = pd.DataFrame(my_dict)

In [ ]:
comp_sample = compare.sample(20)
comp_sample

In [ ]:
comp_sample.plot(kind='bar',figsize=(15,9))
plt.show()

In [ ]:
pd.DataFrame(lm2.coef_, index = X.columns, columns=["Coef"]).sort_values("Coef")

## 4. Implement Ridge Regression

- Import the modul 
- Do not forget to scale the data or use Normalize parameter as True 
- Fit the model 
- Predict the test set 
- Evaluate model performance (use performance metrics for regression) 
- Tune alpha hiperparameter by using [cross validation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html) and determine the optimal alpha value.
- Fit the model and predict again with the new alpha value. 

## Scaling

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Ridge

In [ ]:
from sklearn.linear_model import Ridge, RidgeCV

In [ ]:
ridge_model = Ridge(random_state=42)

In [ ]:
ridge_model.fit(X_train_scaled, y_train)

In [ ]:
train_val(ridge_model, X_train_scaled, y_train, X_test_scaled, y_test) 
# as seen, no overfitting because Ridge has alreday dealing with it

## Finding best alpha for Ridge

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
alpha_space = np.linspace(0.01, 100, 100)
alpha_space

In [ ]:
ridge_model = Ridge(random_state=42)

param_grid = {'alpha':alpha_space}

ridge_grid_model = GridSearchCV(estimator=ridge_model,
                          param_grid=param_grid,
                          scoring='neg_root_mean_squared_error',
                          cv=10,
                          n_jobs = -1)

In [ ]:
ridge_grid_model.fit(X_train_scaled,y_train)

In [ ]:
#ridge_grid_model.best_estimator_

In [ ]:
ridge_grid_model.best_params_ # best alpha for ridge:1.02

In [ ]:
pd.DataFrame(ridge_grid_model.cv_results_)

In [ ]:
ridge_grid_model.best_index_

In [ ]:
ridge_grid_model.best_score_

In [ ]:
train_val(ridge_grid_model, X_train_scaled, y_train, X_test_scaled, y_test)

In [ ]:
y_pred = ridge_grid_model.predict(X_test_scaled)
rm_R2 = r2_score(y_test, y_pred)
rm_mae = mean_absolute_error(y_test, y_pred)
rm_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
ridge = Ridge(alpha=1.02, random_state=42).fit(X_train_scaled, y_train)

pd.DataFrame(ridge.coef_, index = X.columns, columns=["Coef"]).sort_values("Coef")

## 5. Implement Lasso Regression

- Import the modul 
- Do not forget to scale the data or use Normalize parameter as True(If needed)
- Fit the model 
- Predict the test set 
- Evaluate model performance (use performance metrics for regression) 
- Tune alpha hyperparameter by using [cross validation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html) and determine the optimal alpha value.
- Fit the model and predict again with the new alpha value.
- Compare different evaluation metrics

*Note: To understand the importance of the alpha hyperparameter, you can observe the effects of different alpha values on feature coefficants.*

In [ ]:
from sklearn.linear_model import Lasso, LassoCV

In [ ]:
lasso_model = Lasso(random_state=42)
lasso_model.fit(X_train_scaled, y_train)

In [ ]:
train_val(lasso_model, X_train_scaled, y_train, X_test_scaled, y_test)

## Finding best alpha for Lasso

In [ ]:
lasso_model = Lasso(random_state=42)

param_grid = {'alpha':alpha_space}

lasso_grid_model = GridSearchCV(estimator=lasso_model,
                          param_grid=param_grid,
                          scoring='neg_root_mean_squared_error',
                          cv=10,
                          n_jobs = -1)

In [ ]:
lasso_grid_model.fit(X_train_scaled,y_train)

In [ ]:
lasso_grid_model.best_params_  # best alpha for lasso : 1.02

In [ ]:
lasso_grid_model.best_score_

In [ ]:
train_val(lasso_grid_model, X_train_scaled, y_train, X_test_scaled, y_test)

In [ ]:
y_pred = lasso_grid_model.predict(X_test_scaled)
lasm_R2 = r2_score(y_test, y_pred)
lasm_mae = mean_absolute_error(y_test, y_pred)
lasm_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
lasso = Lasso(alpha=1.02, random_state=42).fit(X_train_scaled, y_train)
pd.DataFrame(lasso.coef_, index = X.columns, columns=["Coef"]).sort_values("Coef")

## 6. Implement Elastic-Net

- Import the modul 
- Do not forget to scale the data or use Normalize parameter as True(If needed)
- Fit the model 
- Predict the test set 
- Evaluate model performance (use performance metrics for regression) 
- Tune alpha hyperparameter by using [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) and determine the optimal alpha value.
- Fit the model and predict again with the new alpha value.
- Compare different evaluation metrics

In [ ]:
from sklearn.linear_model import ElasticNet

In [ ]:
elastic_model = ElasticNet(random_state=42)
elastic_model.fit(X_train_scaled,y_train)

In [ ]:
train_val(elastic_model, X_train_scaled, y_train, X_test_scaled, y_test)

## Finding best alpha and l1_ratio for ElasticNet

In [ ]:
elastic_model = ElasticNet(random_state=42)

In [ ]:
param_grid = {'alpha':[1.02, 2,  3, 4, 5, 7, 10, 11],
              'l1_ratio':[.5, .7, .9, .95, .99, 1]}

elastic_grid_model = GridSearchCV(estimator=elastic_model,
                          param_grid=param_grid,
                          scoring='neg_root_mean_squared_error',
                          cv=10,
                          n_jobs = -1)

In [ ]:
elastic_grid_model.fit(X_train_scaled,y_train)

In [ ]:
elastic_grid_model.best_params_   # best alpha and l1_ratio for elasticnet: 1.02 and  1

In [ ]:
elastic_grid_model.best_score_

In [ ]:
train_val(elastic_grid_model, X_train_scaled, y_train, X_test_scaled, y_test)

In [ ]:
y_pred = elastic_grid_model.predict(X_test_scaled)
em_R2 = r2_score(y_test, y_pred)
em_mae = mean_absolute_error(y_test, y_pred)
em_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

## Feature İmportance

In [ ]:
from yellowbrick.model_selection import FeatureImportances
from yellowbrick.features import RadViz


viz = FeatureImportances(Lasso(alpha=1.02), labels=X_train.columns)
visualizer = RadViz(size=(720, 3000))
viz.fit(X_train_scaled, y_train)
viz.show();

In [ ]:
df_new = df2[["make_model", "hp_kW", "km","age", "price", "Gearing_Type"]]

In [ ]:
df_new

In [ ]:
df_new.head()

In [ ]:
df_new[df_new["make_model"] == "Audi A2"]

In [ ]:
df_new.drop(index=[2614], inplace =True)

In [ ]:
df_new = df_new[~(df_new.price>35000)]

In [ ]:
df_new = pd.get_dummies(df_new)
df_new

In [ ]:
len(df3)

In [ ]:
X = df_new.drop(columns = ["price"])
y= df_new.price

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
lasso_model = Lasso(random_state=42)

param_grid = {'alpha':alpha_space}

lasso_final_model = GridSearchCV(estimator=lasso_model,
                          param_grid=param_grid,
                          scoring='neg_root_mean_squared_error',
                          cv=10,
                          n_jobs = -1)

In [ ]:
lasso_final_model.fit(X_train_scaled, y_train)

In [ ]:
lasso_final_model.best_params_  # best alpha for lasso with 6 features : 0.01

In [ ]:
lasso_final_model.best_score_

In [ ]:
train_val(lasso_final_model, X_train_scaled, y_train, X_test_scaled, y_test)
# with 6 fetures, r2 score:%86 and %12 error

In [ ]:
2230/df_new.price.mean() # error

In [ ]:
y_pred = lasso_final_model.predict(X_test_scaled)
fm_R2 = r2_score(y_test, y_pred)
fm_mae = mean_absolute_error(y_test, y_pred)
fm_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

## 7. Visually Compare Models Performance In a Graph

In [ ]:
scores = {"linear_m": {"r2_score": lm_R2 , 
 "mae": lm_mae, 
 "rmse": lm_rmse},

 "ridge_m": {"r2_score": rm_R2, 
 "mae": rm_mae,
 "rmse": rm_rmse},
    
 "lasso_m": {"r2_score": lasm_R2, 
 "mae": lasm_mae, 
 "rmse": lasm_rmse},

 "elastic_m": {"r2_score": em_R2, 
 "mae": em_mae, 
 "rmse": em_rmse},
         
 "final_m": {"r2_score": fm_R2, 
 "mae": fm_mae , 
 "rmse": fm_rmse}}
scores = pd.DataFrame(scores).T
scores

In [ ]:
for i in scores:
    print(i)

In [ ]:
#metrics = scores.columns
for i, j in enumerate(scores):
    plt.figure(i)
    if j == "r2_score":
        ascending = False
    else:
        ascending = True
    compare = scores.sort_values(by=j, ascending=ascending)
    ax = sns.barplot(x = compare[j] , y= compare.index)
    for p in ax.patches:
            width = p.get_width()                        # get bar length
            ax.text(width,                               # set the text at 1 unit right of the bar
                    p.get_y() + p.get_height() / 2,      # get Y coordinate + X coordinate / 2
                    '{:.4f}'.format(width),             # set variable to display, 2 decimals
                    ha = 'left',                         # horizontal alignment
                    va = 'center') 

## Prediction new observation

In [ ]:
# X = df_new.drop(columns = ["price"])
# y= df_new.price

In [ ]:
# final_scaler = MinMaxScaler()
# final_scaler.fit(X) # verinin tammamını scaling ediyoruz, train ve testi ayrı ayrı yaplıyoruz çünkü artık modeli deploy edicez
# X_scaled = final_scaler.transform(X)

In [ ]:
X = df_new.drop(columns = ["price"])
y= df_new.price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
final_scaler = MinMaxScaler()
final_scaler.fit(X) # verinin tammamını scaling ediyoruz, train ve testi ayrı ayrı yaplıyoruz çünkü artık modeli deploy edicez
X_scaled = final_scaler.transform(X)
lasso_model = Lasso(random_state=42)

param_grid = {'alpha':alpha_space}

final_model = GridSearchCV(estimator=lasso_model,
                          param_grid=param_grid,
                          scoring='neg_root_mean_squared_error',
                          cv=10,
                          n_jobs = -1)
final_model.fit(X_scaled,y)
filename = 'finalized_model'
pickle.dump(final_model, open(filename, 'wb'))

In [ ]:
# final_model.fit(X_scaled,y)


In [ ]:
# final_model.best_estimator_

In [ ]:
my_dict = {
    "hp_kW": 66,
    "age": 2,
    "km": 17000,
    "make_model": 'Audi A3',
    "Gearing_Type": "Automatic"
}

In [ ]:
my_dict = pd.DataFrame([my_dict])
my_dict

In [ ]:
X.head(1)

In [ ]:
my_dict = pd.get_dummies(my_dict)
my_dict

In [ ]:
my_dict = my_dict.reindex(columns = X.columns, fill_value=0) # yeni sunacağımız veriyi modeldeki sütun düzenine göre ayarlıyoruz
my_dict

In [ ]:
X.head(1)

In [ ]:
my_dict = final_scaler.transform(my_dict)  # yeni veriyi scaling ediyoruz
my_dict

In [ ]:
final_model.predict(my_dict) # bu verilere sahip araba 19559$ eder

___

<p style="text-align: center;"><img src="https://docs.google.com/uc?id=1lY0Uj5R04yMY3-ZppPWxqCr5pvBLYPnV" class="img-fluid" alt="CLRSWY"></p>

___